In [ ]:
import os
import asyncio
import edge_tts
import pygame
import io
import threading
import queue
from openai import OpenAI
from thai_tokenizer import Tokenizer

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-JoABD96Rui1GWSrpJ6h78lTDEkbQG386TWBYZWLPQHoWewkU"
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://api.opentyphoon.ai/v1",
)

# Initialize pygame mixer and Thai tokenizer
pygame.mixer.init()
tokenizer = Tokenizer()

# Create a queue for audio chunks
audio_queue = queue.Queue()

# List of EdgeTTS voices to try
VOICES = ["th-TH-NiwatNeural", "th-TH-PremwadeeNeural", "en-US-ChristopherNeural"]

async def text_to_speech(text):
    for voice in VOICES:
        try:
            communicate = edge_tts.Communicate(text, voice)
            audio_data = io.BytesIO()
            async for chunk in communicate.stream():
                if chunk["type"] == "audio":
                    audio_data.write(chunk["data"])
            audio_data.seek(0)
            return audio_data
        except edge_tts.exceptions.NoAudioReceived:
            print(f"Failed to generate audio with voice {voice}. Trying next voice...")
    
    print("All voices failed. Unable to generate audio.")
    return None

def play_audio():
    while True:
        audio = audio_queue.get()
        if audio is None:
            break
        if audio != "skip":
            pygame.mixer.music.load(audio)
            pygame.mixer.music.play()
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)
        audio_queue.task_done()

async def stream_and_speak():
    buffer = ""
    stream = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[
            {
                "role": "user",
                "content": "วันนี้กินข้าวกับอะไรดี",
            }
        ],
        max_tokens=512,
        temperature=0.6,
        top_p=0.95,
        stream=True,
    )

In [1]:
from thai_tokenizer import Tokenizer
tokenizer = Tokenizer()
tokenizer('iPad Mini 256GB เครื่องไทย') #> 'iPad Mini 256GB เครื่อง ไทย'
tokenizer.split('เครื่องไทย') #> ['เครื่อง', 'ไทย']

['เครื่อง', 'ไทย']

In [4]:
import os
import asyncio
import edge_tts
import pygame
import io
import threading
import queue
from openai import OpenAI
from thai_tokenizer import Tokenizer

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-JoABD96Rui1GWSrpJ6h78lTDEkbQG386TWBYZWLPQHoWewkU"
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://api.opentyphoon.ai/v1",
)

async def get_response_and_tokenize():
    stream = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[
            {
                "role": "user",
                "content": "วันนี้กินข้าวกับอะไรดี",
            }
        ],
        max_tokens=512,
        temperature=0.6,
        top_p=0.95,
        stream=True,
    )

    response_text = ""
    tokenizer = Tokenizer()

    # Collect and tokenize the response as it streams
    async for chunk in stream:
        if "choices" in chunk:
            chunk_text = chunk["choices"][0]["delta"].get("content", "")
            response_text += chunk_text
            print(f"Received chunk: {chunk_text}")

            # Tokenize the current chunk
            tokens = tokenizer.split(chunk_text)
            print(f"Tokens: {tokens}")

    print("\nFinal Response:")
    print(response_text)
    return response_text

async def main():
    await get_response_and_tokenize()

# If inside an environment with an active event loop (like Jupyter), do this:
if __name__ == "__main__":
    try:
        asyncio.get_running_loop().run_until_complete(main())
    except RuntimeError:  # If no running loop, use asyncio.run()
        asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [5]:
import os
import asyncio
from openai import OpenAI
from thai_tokenizer import Tokenizer

# Set the OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-JoABD96Rui1GWSrpJ6h78lTDEkbQG386TWBYZWLPQHoWewkU"  # Replace with your actual API key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://api.opentyphoon.ai/v1",
)

async def get_response_and_tokenize():
    # Create a stream from the LLM
    stream = client.chat.completions.create(
        model="typhoon-v1.5x-70b-instruct",
        messages=[
            {
                "role": "user",
                "content": "วันนี้กินข้าวกับอะไรดี",
            }
        ],
        max_tokens=512,
        temperature=0.6,
        top_p=0.95,
        stream=True,
    )

    response_text = ""
    tokenizer = Tokenizer()

    # Collect and tokenize the response as it streams
    async for chunk in stream:
        if "choices" in chunk:
            chunk_text = chunk["choices"][0]["delta"].get("content", "")
            response_text += chunk_text
            print(f"Received chunk: {chunk_text}")

            # Tokenize the current chunk
            tokens = tokenizer.split(chunk_text)
            print(f"Tokens: {tokens}")

    print("\nFinal Response:")
    print(response_text)
    return response_text

async def main():
    await get_response_and_tokenize()

# Handling the event loop
if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
        loop.run_until_complete(main())
    except RuntimeError:
        asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop